In [65]:
from tools.data import get_datasets 
from sklearn import naive_bayes
import features
import numpy as np

# dataset : http://www.mindbigdata.com/opendb/
# Mfcc : https://github.com/jameslyons/python_speech_features

train, test = get_datasets()

Reading data file


In [66]:
#mtrain et mtest -> Ensemble de données avec juste des signaux valides
#btrain et btest -> Ensemble donn/es avec classe -1 (pas un signal) et 1 (est un signal)
mtrain, mtest = remove_class(train, -1),remove_class(test, -1)
btrain, btest = replace_classes(train, [i for i in range(10)],1),replace_classes(test, [i for i in range(10)],1)

print(mtrain)
print(btrain)

[[ 476.  509.  509. ...,    0.    0.    4.]
 [ 442.  493.  501. ...,    0.    0.    4.]
 [ 476.  509.  520. ...,    0.    0.    2.]
 ..., 
 [ 476.  517.  518. ...,    0.    0.    4.]
 [ 476.  509.  517. ...,    0.    0.    2.]
 [ 425.  499.  534. ...,    0.    0.    3.]]
[[ 476.  509.  509. ...,    0.    0.    1.]
 [ 460.  528.  520. ...,    0.    0.   -1.]
 [ 460.  500.  496. ...,    0.    0.   -1.]
 ..., 
 [ 476.  517.  518. ...,    0.    0.    1.]
 [ 476.  509.  517. ...,    0.    0.    1.]
 [ 425.  499.  534. ...,    0.    0.    1.]]


todo :

- changer la variable nb_capteur pour nb_signal.
- calculer dynamiquement la taille des frames à utiliser et le step, pour pouvoir avoir un nombre fixe de ceptrum pour un signal.
- feature mfcc_std
- Tester d'autre algorithmes de classification

In [82]:
#Mu - ~220Hz - 4 capteurs
#MW - ~512Hz -
#EP - ~128Hz
#IN - ~128Hz
#Each sample of the dataset last about 2 secs.

def getmfcc2(data, nbcapteurs=4, buffer_size=512, numcep=13, numframes=94):
    """ 
    Dynamically calculate the window size and window step for each signal to obtain a given number of frame.
    
    unfinished....
    """
    
    mfcc_features = np.zeros((data.shape[0], nbcapteurs, numframe, numcep)) #hardcodé la valeur pour le nombre de frame...
    #fbank_feat = np.zeros((data.shape[0], numbank))
    print('shape', mfcc_features.shape)
    
    for x, d in enumerate(data):    
        length = int(d[0])
        #reversing the computation of the number of frame
        frame_len = length / numframes #=winlen*samplerate
        #frame_step =  #=winstep*sample rate
        print('expected',numframes,'calculated',1+ int(math.ceil((1.0*length - frame_len)/frame_step)))
#      numframes = 1 + int(math.ceil((1.0*slen - frame_len)/frame_step))
        
        for i in range(nbcapteurs):
            pass

    return mfcc_features


def getmfcc(data, nbcapteurs=4, buffer_size=512, numcep=13):
    """ 
    Consider each signal as a whole (including the zero-padding). 
    While this technique return a fixed number of for all signal, this causes a severed distorsion in the resonnance because of the zero padding.
    
    returns a tensor (nb_examples x nb_capteurs x numframes x numcep)
    """
    
    mfcc_features = np.zeros((data.shape[0], nbcapteurs, 101, numcep)) #hardcodé la valeur pour le nombre de frame...
    #fbank_feat = np.zeros((data.shape[0], numbank))
    print('shape', mfcc_features.shape)
    
    for x, d in enumerate(data):    
        length = int(d[0])
        
        for i in range(nbcapteurs):

            #signal = d[1+i*buffer_size:1+i*buffer_size + length]# I do not get a fix number of frame if I do not use the zero-padding.
            #print(length)
            signal = d[1+i*buffer_size: 1+(i+1)*buffer_size]
            mfcc_features[x,i,:,:] = features.mfcc(signal, buffer_size, numcep=numcep)
    
    return mfcc_features


def most_resonnant_by_signal(data,nbcapteurs=4,buffer_size=512, numcep=30):
    """
    Returns a matrix of shape ( nb of examples x nb of signals)
    where each element at index i,j is the index of the most resonnant ceptrum by signal.
    """

    feat = np.zeros((data.shape[0], nbcapteurs))
    
    for x, d in enumerate(data): 
        length = int(d[0])
        
        for i in range(nbcapteurs):

            signal = d[1+i*buffer_size:1+i*buffer_size + length]# I do not get a fix number of frame if I do not use the zero-padding.
            feat[x,i] = np.argmax(features.mfcc(signal, buffer_size, numcep=numcep))
    
    return feat
    

def most_resonnance_std(mfcc):
    """
    Take in input a 4d array of which of shape (nb_examples x nb_capteurs x num_frames x num_ceptrum)
    this function supports the possibility an different number frame by signal
    returns
    
    
    not finished.
    """
              
    print(mfcc.shape)
    i,c,f,x = mfcc.shape
    rez = np.apply_along_axis(np.argmax,3,mfcc)
              
    print(rez)
    return rez



In [83]:
def results(data, nbcapteurs=4, buffer_size=512, numcep=13, numframes=94):
    """ data : longueur | 512 mesures pour capteur 1 | 512 mesures pour capteur 2 | 512 mesures pour capteur 3 | 512 mesures pour capteur 4 | Chiffre vu | """
    return data[:,-1]

def remove_class(data, target, copy=True):
    """
    Remove all examples which class == target.
    assumes that the class is at the last index of the data
    """
    if copy :
        return np.array(data[data[:,-1] != -1],copy=True)
    else:
        return data[data[:,-1] != -1]

def replace_classes(data, to_replace, replace_by, copy = True):
    """
    For all examples which class in to_replace. Chance class for replace_by
    assumes that the class is at the last index of the data
    """
    if copy:
        data = np.array(data,copy=True)
    
    for x in data:
        if x[-1] in to_replace:
            x[-1] = replace_by
    
    return data

def quicktest(train,train_result,test,test_result,model):
    """Train and test model and print relative information"""
    print("Training "+str(train.shape[0])+" examples...")
    model.fit(train,train_result)
    print("Testing on "+str(test.shape[0])+" examples...")
    y = model.predict(test)
    print("Errors maded " +str(np.sum(test_result != y)))
    print("Predictions : ")
    print(y)
    print("Reality check : ")
    print(test_result)


La prochaine section teste les données brutes dans un classifieur de Bayes naïf. 
Étrangement, le procédé donne 100% pour tous les types de classification. 

In [86]:


if False: # if True, it will not train and test on the whole dataset. this is for developping purpose
    train_cut = 10 #takes only the 'train_cut' first example in training set.
    test_cut = 10000#takes only the 'train_cut' first example in training set.

    quicktest(btrain[:train_cut],results(btrain[:train_cut]), btest[:test_cut],results(btest[:test_cut]),naive_bayes.GaussianNB())
    quicktest(mtrain[:train_cut],results(mtrain[:train_cut]), mtest[:test_cut],results(mtest[:test_cut]),naive_bayes.GaussianNB())
    quicktest(train[:train_cut],results(train[:train_cut]), test[:test_cut],results(test[:test_cut]),naive_bayes.GaussianNB())
elif False :
    #Classification binaire
    quicktest(btrain,results(btrain), btest, results(btest),naive_bayes.GaussianNB())
    #Classification multiclasse sans -1
    quicktest(mtrain,results(mtrain), mtest, results(mtest),naive_bayes.GaussianNB())
    #Classification multiclasse avec -1
    quicktest(train, results(train), test, results(test), naive_bayes.GaussianNB())
    


Training 30000 examples...
Testing on 10983 examples...
Errors maded 2930
Predictions : 
[ 1.  1.  1. ...,  1.  1.  1.]
Reality check : 
[-1.  1.  1. ..., -1.  1. -1.]
Training 21827 examples...
Testing on 8053 examples...
Errors maded 7182
Predictions : 
[ 6.  6.  8. ...,  6.  5.  5.]
Reality check : 
[ 7.  7.  0. ...,  4.  1.  0.]
Training 30000 examples...
Testing on 10983 examples...
Errors maded 8053
Predictions : 
[-1. -1. -1. ..., -1. -1. -1.]
Reality check : 
[-1.  7.  7. ..., -1.  0. -1.]


La prochaine section teste le vecteur de feature qu'est le vecteur des indexes du ceptrum le plus résonnant pour chaque capteurs. Le feature vector est donc de taille 4 pour le casque Muse.

###Classification binaire
Training 30000 examples...
Testing on 10983 examples...
Errors maded 2930
-> 26.678 % d'erreurs

###Classification multiclasse sans -1
Training 21827 examples...
Testing on 8053 examples...
Errors maded 7182
-> 89.18% d'erreurs

###Classification multiclasse avec -1
Training 30000 examples...
Testing on 10983 examples...
Errors maded 8053
-> 73.32% d'erreurs


In [ ]:
if False:
    train_cut = 10 #takes only the 'train_cut' first example in training set.
    test_cut = 10000#takes only the 'train_cut' first example in training set.

    quicktest(most_resonnant_by_signal(btrain[:train_cut]),results(btrain[:train_cut]), most_resonnant_by_signal(btest[:test_cut]),results(btest[:test_cut]),naive_bayes.GaussianNB())
    quicktest(most_resonnant_by_signal(mtrain[:train_cut]),results(mtrain[:train_cut]), most_resonnant_by_signal(mtest[:test_cut]),results(mtest[:test_cut]),naive_bayes.GaussianNB())
    quicktest(most_resonnant_by_signal(train[:train_cut]),results(train[:train_cut]), most_resonnant_by_signal(test[:test_cut]),results(test[:test_cut]),naive_bayes.GaussianNB())    

else :
    #Classification binaire
    quicktest(most_resonnant_by_signal(btrain),results(btrain), most_resonnant_by_signal(btest), results(btest),naive_bayes.GaussianNB())
    #Classification multiclasse sans -1
    quicktest(most_resonnant_by_signal(mtrain),results(mtrain), most_resonnant_by_signal(mtest), results(mtest),naive_bayes.GaussianNB())
    #Classification multiclasse avec -1
    quicktest(most_resonnant_by_signal(train), results(train), most_resonnant_by_signal(test), results(test), naive_bayes.GaussianNB())


In [70]:
#pris d'une démo.
class gauss_mv:

    def __init__(self,n_dims,cov_type="isotropique"):
        self.n_dims = n_dims
        self.mu = np.zeros((1,n_dims))
        self.cov_type = cov_type
        if cov_type=="isotropique":
            self.sigma_sq = 1.0
        elif cov_type=="diagonale":
            self.sigma_sq = numpy.ones(n_dims)
        elif cov_type=="full":
            self.cov = np.ones((n_dims,n_dims))

    # Pour un ensemble d'entrainement, la fonction devrait calculer l'estimateur par MV de la moyenne et de la matrice de covariance
    def train(self, train_data):
        self.mu = np.mean(train_data, axis=0)
        if self.cov_type == "isotropique":
            self.sigma_sq = np.sum((train_data - self.mu)**2.0) / (self.n_dims * train_data.shape[0])
        elif self.cov_type == "diagonale":
            self.sigma_sq =  numpy.sum((train_data - self.mu) ** 2.0, axis = 0) / train_data.shape[0] 
        elif self.cov_type == "full":
            self.cov = np.cov(np.transpose(train_data))
        pass
    
    # Retourne un vecteur de taille nb. ex. de test contenant les log
    # probabilités de chaque exemple de test sous le modèle.    
    def compute_predictions(self, test_data):

        if self.cov_type == "isotropique":
            # log(constante de normalisation)
            c = -self.n_dims * np.log(2*np.pi)/2.0 - self.n_dims*np.log(self.sigma_sq)/2.0
            # il faut calculer la valeur de la log-probabilite de chaque exemple
            # de test sous le modele determine par mu et sigma_sq. le vecteur
            # des probabilites est/sera log_prob
            log_prob = c - np.sum((test_data -  self.mu)**2.0,axis=1) / (2.0 * self.sigma_sq)
        elif self.cov_type == "diagonale":
            # on prend le produit du vecteur représentant la diagonale (np.prod(self.sigma)
            c = -self.n_dims * np.log(2*np.pi)/2.0 - self.n_dims*np.log(np.prod(self.sigma_sq))/2.0
            # on somme sur l'axe 1 après avoir divisé par sigma puisque celui ci aussi est
            # de dimension d
            log_prob = c - np.sum((test_data -  self.mu)**2.0/ (2.0 * self.sigma_sq),axis=1)
        elif self.cov_type == "full":
            c = -self.n_dims * np.log(2.0*np.pi)/2.0
            det = np.linalg.det(self.cov)
            c += np.log(det)/2.0
            
            dmu = test_data-self.mu
            inv = np.linalg.inv(self.cov)
            
            dxs = np.dot(dmu,inv)
            dxsx = np.sum(dxs*dmu,axis=1)
            log_prob = c - dxsx
        return log_prob

#Pris de la démo 5.

class classif_bayes:

    def __init__(self,modeles_mv, priors):
        self.modeles_mv = modeles_mv
        self.priors = priors
        if len(self.modeles_mv) != len(self.priors):
            print 'Le nombre de modeles MV doit etre egale au nombre de priors!'
        
        self.n_classes = len(self.modeles_mv)
                                                            
    # Retourne une matrice de taille nb. ex. de test x nombre de classes contenant les log
    # probabilités de chaque exemple de test sous chaque modèle MV. 
    def compute_predictions(self, test_data, eval_by_group=False):
        log_pred = np.empty((test_data.shape[0],self.n_classes))

        for i in range(self.n_classes):
            # ici il va falloir utiliser modeles_mv[i] et priors pour remplir
            # chaque colonne de log_pred (c'est plus efficace de faire tout une
            # colonne a la fois)
            
            log_pred[:,i] = self.modeles_mv[i].compute_predictions(test_data) +  self.priors[i]

        return log_pred

In [71]:

#trier les exemples pour classificateur de Bayes
def sort_examples(data)
    sorted_data = {i:set() for i in [-1] + list(range(10))}

    for ex in data:
        sorted_data[ex[-1]].append(ex[:-1])
    
    return sorted_data
            
#sorting data and training models

s = sort_examples(train)
models = []
for i in list(range(10)) + [-1]  :
    model_classe1=gauss_mv(len(train_cols))
    model_classe1.train(iris_train1)
    
#Computing prior. should we assume equiprobability for every result that is not -1
priors = []    


SyntaxError: invalid syntax (<ipython-input-71-147e04bbe5ec>, line 3)